In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 4827}


4827

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 8480}


8480

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,379d9a3050314f1692130a596e5c2cb9,2volmar,2022-03-09T16:44:30.486000Z,c87afecd004a4da0b60f0351aa28bb34,5,Fontin,Bertho
1,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-03-09T16:31:01.548000Z,f9c8e6119dee4fd0a8997ac11e0aa70d,3,Louimene,Jules
2,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-03-09T16:28:58.584000Z,d74f77cf83aa4576ab6939a666c4e9aa,2,Marie-Leonie,Joseph
3,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-03-09T16:27:30.239000Z,bce303b355a948f89847e76fedd38618,2,Marie Elicile,Louis
4,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-03-09T16:25:09.631000Z,9d94f75dafe34404989be6b227e4b3ac,3,Marisens Carde,Calixte
...,...,...,...,...,...,...,...
4675,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
4676,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
4677,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
4678,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


4680

In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
8475,379d9a3050314f1692130a596e5c2cb9,69bd6a37-1336-434e-9f9f-7746dd3ca12f,14,2,non,non,c87afecd004a4da0b60f0351aa28bb34,child
8476,379d9a3050314f1692130a596e5c2cb9,bf738d91-6fc8-4937-ae47-ed8abc49fa21,11,2,non,non,c87afecd004a4da0b60f0351aa28bb34,child
8477,379d9a3050314f1692130a596e5c2cb9,1bbf628b-68a6-4c52-8fca-33f4a98df86e,9,2,non,non,c87afecd004a4da0b60f0351aa28bb34,child
8478,379d9a3050314f1692130a596e5c2cb9,b90cdd27-de43-4632-bd72-9c8fed4a6176,6,1,non,non,c87afecd004a4da0b60f0351aa28bb34,child


8480

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [8]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.sum(numeric_only=True, axis=0)
result_hivmuso.reset_index(inplace=True)
result_hivmuso
result_hivmuso.qty.sum()

,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",50
1,"(Gustave, firma.gustave@carisfoundationintl.org)",199
2,"(June007, jeanrony.valbrun@carisfoundationintl...",57
3,"(jfodney17, fodney.joseph@carisfoundationintl....",114
4,"(josy22, josy.serafils@carisfoundationintl.org)",65
5,"(judesonvOLMAR, judeson.volmar@carisfoundation...",41
6,"(marcakernstdubois, kernst.dubois@carisfoundat...",71
7,"(suzanne, suzanne.clerge@carisfoundationintl.org)",116
8,Total,713


1426

In [9]:
total = DataFrame(
    {
        "total":[result_hivmuso.qty.sum()]
    }
)

total

,total
0,1426
